In [2]:
import mlflow

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("DSPy")

<Experiment: artifact_location='mlflow-artifacts:/374362034103955121', creation_time=1741686562632, experiment_id='374362034103955121', last_update_time=1741686562632, lifecycle_stage='active', name='DSPy', tags={}>

In [3]:
mlflow.dspy.autolog()

In [8]:
import os
import tempfile
from datasets import load_dataset
from typing import Dict, Any, List
import dspy

def load_conll_dataset() -> dict:
    """
    Loads the CoNLL-2003 dataset into train, validation, and test splits.
    
    Returns:
        dict: Dataset splits with keys 'train', 'validation', and 'test'.
    """
    with tempfile.TemporaryDirectory() as temp_dir:
        # Use a temporary Hugging Face cache directory for compatibility with certain hosted notebook
        # environments that don't support the default Hugging Face cache directory
        os.environ["HF_DATASETS_CACHE"] = temp_dir
        return load_dataset("conll2003", trust_remote_code=True)

def extract_people_entities(data_row: Dict[str, Any]) -> List[str]:
    """
    Extracts entities referring to people from a row of the CoNLL-2003 dataset.
    
    Args:
        data_row (Dict[str, Any]): A row from the dataset containing tokens and NER tags.
    
    Returns:
        List[str]: List of tokens tagged as people.
    """
    return [
        token
        for token, ner_tag in zip(data_row["tokens"], data_row["ner_tags"])
        if ner_tag in (1, 2)  # CoNLL entity codes 1 and 2 refer to people
    ]

def prepare_dataset(data_split, start: int, end: int) -> List[dspy.Example]:
    """
    Prepares a sliced dataset split for use with DSPy.
    
    Args:
        data_split: The dataset split (e.g., train or test).
        start (int): Starting index of the slice.
        end (int): Ending index of the slice.
    
    Returns:
        List[dspy.Example]: List of DSPy Examples with tokens and expected labels.
    """
    return [
        dspy.Example(
            tokens=row["tokens"],
            expected_extracted_people=extract_people_entities(row)
        ).with_inputs("tokens")
        for row in data_split.select(range(start, end))
    ]

# Load the dataset
dataset = load_conll_dataset()

# Prepare the training and test sets
train_set = prepare_dataset(dataset["train"], 0, 50)
test_set = prepare_dataset(dataset["test"], 0, 200)

train_set[1]

Example({'tokens': ['Peter', 'Blackburn'], 'expected_extracted_people': ['Peter', 'Blackburn']}) (input_keys={'tokens'})

In [5]:
from typing import List

class PeopleExtraction(dspy.Signature):
    """
    Extract contiguous tokens referring to specific people, if any, from a list of string tokens.
    Output a list of tokens. In other words, do not combine multiple tokens into a single value.
    """
    tokens: list[str] = dspy.InputField(desc="tokenized text")
    extracted_people: list[str] = dspy.OutputField(desc="all tokens referring to specific people extracted from the tokenized text")

people_extractor = dspy.ChainOfThought(PeopleExtraction)

In [6]:
lm = dspy.LM(model="openai/gpt-4o-mini")
dspy.settings.configure(lm=lm)

In [9]:
def extraction_correctness_metric(example: dspy.Example, prediction: dspy.Prediction, trace=None) -> bool:
    """
    Computes correctness of entity extraction predictions.
    
    Args:
        example (dspy.Example): The dataset example containing expected people entities.
        prediction (dspy.Prediction): The prediction from the DSPy people extraction program.
        trace: Optional trace object for debugging.
    
    Returns:
        bool: True if predictions match expectations, False otherwise.
    """
    return prediction.extracted_people == example.expected_extracted_people

evaluate_correctness = dspy.Evaluate(
    devset=test_set,
    metric=extraction_correctness_metric,
    num_threads=24,
    display_progress=True,
    display_table=True
)

In [10]:
evaluate_correctness(people_extractor, devset=test_set)

Average Metric: 181.00 / 200 (90.5%): 100%|██████████| 200/200 [00:16<00:00, 12.32it/s]

2025/03/13 14:25:55 INFO dspy.evaluate.evaluate: Average Metric: 181 / 200 (90.5%)


,tokens,expected_extracted_people,reasoning,extracted_people,extraction_correctness_metric
0,"[SOCCER, -, JAPAN, GET, LUCKY, WIN, ,, CHINA, IN, SURPRISE, DEFEAT...",[CHINA],The tokens provided do not contain any specific names of people. T...,[],
1,"[Nadim, Ladki]","[Nadim, Ladki]","The tokens ""Nadim"" and ""Ladki"" refer to specific individuals. They...","[Nadim, Ladki]",✔️ [True]
2,"[AL-AIN, ,, United, Arab, Emirates, 1996-12-06]",[],The provided tokens do not contain any references to specific peop...,[],✔️ [True]
3,"[Japan, began, the, defence, of, their, Asian, Cup, title, with, a...",[],The provided tokens do not contain any specific names of people. T...,[],✔️ [True]
4,"[But, China, saw, their, luck, desert, them, in, the, second, matc...",[],"In the provided tokens, ""China"" and ""Uzbekistan"" are the only toke...",[],✔️ [True]
...,...,...,...,...,...
195,"['The', 'Wallabies', 'have', 'their', 'sights', 'set', 'on', 'a', ...","[David, Campese]","The tokenized text mentions ""David Campese,"" who is a specific per...","[David, Campese]",✔️ [True]
196,"['The', 'Wallabies', 'currently', 'have', 'no', 'plans', 'to', 'ma...",[],"The text mentions ""the 34-year-old winger,"" which refers to a spec...",[],✔️ [True]
197,"['Campese', 'will', 'be', 'up', 'against', 'a', 'familiar', 'foe',...","[Campese, Rob, Andrew]","The tokens contain references to specific people, namely ""Campese""...","[Campese, Rob, Andrew]",✔️ [True]
198,"['""', 'Campo', 'has', 'a', 'massive', 'following', 'in', 'this', '...","[Campo, Andrew]","The tokenized text mentions ""Andrew"" as a specific person. It is t...",[Andrew],


90.5

[Trace(request_id=24c25c79828f4423931dacf53164a4d5), Trace(request_id=ec0246f88f2b48808b77328976445bd3), Trace(request_id=eb69ad4fab1a42d8ac7069fa70f94e31), Trace(request_id=b8715e69b68d4065b3c5e1c9f4a99dc3), Trace(request_id=34f6b5f258494943a82537a7a58deb1b), Trace(request_id=7c548b24117948e1a0e518a34ff29482), Trace(request_id=7d8e095e8c6c4571963ba7552ebf6662), Trace(request_id=bac79b58ccbf4c668a14802c9273ade1), Trace(request_id=a5c17e28603f4a248f4e2d6a9ead63be), Trace(request_id=85e8c3cd96ca42ce831d992d3b8510b2)]

In [11]:
import mlflow

with mlflow.start_run(run_name="extractor_evaluation"):
    evaluate_correctness = dspy.Evaluate(
        devset=test_set,
        metric=extraction_correctness_metric,
        num_threads=24,
        display_progress=True,
        # To record the outputs and detailed scores to MLflow
        return_all_scores=True,
        return_outputs=True,
    )

    # Evaluate the program as usual
    aggregated_score, outputs, all_scores = evaluate_correctness(people_extractor)

    # Log the aggregated score
    mlflow.log_metric("exact_match", aggregated_score)
    # Log the detailed evaluation results as a table
    mlflow.log_table(
        {
            "Tokens": [example.tokens for example in test_set],
            "Expected": [example.expected_extracted_people for example in test_set],
            "Predicted": outputs,
            "Exact match": all_scores,
        },
        artifact_file="eval_results.json",
    )

Average Metric: 181.00 / 200 (90.5%): 100%|██████████| 200/200 [00:01<00:00, 113.49it/s]

2025/03/13 14:28:54 INFO dspy.evaluate.evaluate: Average Metric: 181 / 200 (90.5%)



🏃 View run extractor_evaluation at: http://localhost:5000/#/experiments/374362034103955121/runs/9a305735ced747d897fb77fc4d195825
🧪 View experiment at: http://localhost:5000/#/experiments/374362034103955121


[Trace(request_id=5c10a2b4f89644d7a536b70156d872dc), Trace(request_id=32931affce454ed3b981a375fe413238), Trace(request_id=277468f416374168b96d987c1e3df3aa), Trace(request_id=07473c288e54446da47170017f9514e3), Trace(request_id=1db84fe235924be2a7db1e52ea3326af), Trace(request_id=cf5694d0169b439d87b544d2e72d7a56), Trace(request_id=c92a86792ecd40d79be059197dbaa69f), Trace(request_id=071e979540774766a2b13241d1d19d88), Trace(request_id=6020059fb36a4911b15b5790dae8942d), Trace(request_id=a449c79c49db4d42a076b8da80d96333)]

In [ ]:
mipro_optimizer = dspy.MIPROv2(
    metric=extraction_correctness_metric,
    auto="medium",
)
optimized_people_extractor = mipro_optimizer.compile(
    people_extractor,
    trainset=train_set,
    max_bootstrapped_demos=4,
    requires_permission_to_run=False,
    minibatch=False
)

2025/03/13 14:30:01 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING MEDIUM AUTO RUN SETTINGS:
num_trials: 25
minibatch: False
num_candidates: 19
valset size: 40

2025/03/13 14:30:01 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/03/13 14:30:01 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/03/13 14:30:01 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=19 sets of demonstrations...


Bootstrapping set 1/19
Bootstrapping set 2/19
Bootstrapping set 3/19


 10%|█         | 1/10 [00:02<00:18,  2.01s/it]